In [298]:
from __future__ import division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, utils
from PIL import Image
from torchvision import transforms
from torch.autograd import Variable
import random
import math
from functools import reduce

import cv2
from glob import glob

import skimage
from skimage.io import imread
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline
from skimage.color import rgb2grey, rgb2hsv, hsv2rgb, grey2rgb, rgba2rgb
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, utils
from PIL import Image
from torchvision import transforms
from torch.autograd import Variable
import random
import math
from functools import reduce

import cv2
from glob import glob

import skimage
from skimage.io import imread
from skimage import img_as_float, img_as_ubyte
from skimage.morphology import reconstruction
from skimage.util import invert
from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from skimage.color import label2rgb
from skimage.filters import gaussian

from skimage.morphology import erosion, dilation, binary_dilation, opening, closing, white_tophat
from skimage.morphology import disk

from sklearn.model_selection import train_test_split
from skimage.morphology import label

from utils import show_img, plot_img_and_hist

In [260]:
transform = reload(transform)

In [307]:
#%%writefile dataset.py
#%pycat dataset.py

from __future__ import division
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, utils
from PIL import Image
from torch.autograd import Variable
import random
import math
from functools import reduce

from glob import glob

import skimage
from skimage.io import imread
from skimage import img_as_float, img_as_ubyte

import sklearn


class NucleusDataset(Dataset):
    """Nucleus dataset."""

    @staticmethod
    def read_and_stack(in_img_list):
        # return np.sum(np.stack([i*(imread(c_img)>0) for i, c_img in
        # enumerate(in_img_list, 1)], 0), 0)
        r = (reduce(
                np.bitwise_or, [
                    np.asarray(Image.open(c_img)) for c_img in iter(in_img_list)])).astype(np.uint8)
        #r = r / r.max()
        return r

    @staticmethod
    def read_image(in_img_list):
        #img = img_as_float(rgb2hsv(rgba2rgb(io.imread(in_img_list[0]))))
        img = Image.open(in_img_list[0])
        return np.array(img.convert('RGB')), img.size

    def __init__(
            self,
            root_dir=None,
            stage_name='stage1',
            group_name='train',
            transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """

        self.root_dir = root_dir
        self.transform = transform

        if root_dir is None:
            return

        all_images = glob(
            os.path.join(
                root_dir,
                stage_name +
                '_*',
                '*',
                '*',
                '*'))
        
        img_df = pd.DataFrame({'path': all_images})

        def img_id(in_path): return in_path.split('/')[-3]

        def img_type(in_path): return in_path.split('/')[-2]

        def img_group(in_path): return in_path.split('/')[-4].split('_')[1]

        def img_stage(in_path): return in_path.split('/')[-4].split('_')[0]
        img_df['id'] = img_df['path'].map(img_id)
        img_df['type'] = img_df['path'].map(img_type)
        img_df['group'] = img_df['path'].map(img_group)
        img_df['stage'] = img_df['path'].map(img_stage)
        self.img_df = img_df

        data_df = img_df.query('group=="%s"' % group_name)
        data_rows = []
        group_cols = ['stage', 'id']
        for n_group, n_rows in data_df.groupby(group_cols):
            c_row = {
                col_name: col_value for col_name,
                col_value in zip(
                    group_cols,
                    n_group)}
            c_row['masks'] = n_rows.query('type == "masks"')[
                'path'].values.tolist()
            c_row['images'] = n_rows.query('type == "images"')[
                'path'].values.tolist()
            data_rows += [c_row]

        data_df = pd.DataFrame(data_rows)

        ret = data_df['images'].map(self.read_image)
        #(data_df['images'], data_df['format'], data_df['mode'], data_df['size']) = ([x[i] for x in ret] for i in range(4))
        (data_df['images'], data_df['size']) = ([x[i] for x in ret] for i in range(2))
    
        data_df['masks'] = data_df['masks'].map(
            self.read_and_stack).map(
            lambda x: x.astype(int))

        # print data_df.describe()
        self.data_df = data_df

    def __len__(self):
        return self.data_df.shape[0]

    def __getitem__(self, idx):

        sample = self.data_df["images"].iloc[idx]
        masks =  self.data_df["masks"].iloc[idx]
        # insert one dummy channel, to be consistent with output
        masks = np.expand_dims(masks, 2)
        if self.transform:
            sample, masks = self.transform(sample, masks)

        return sample, masks

# https://discuss.pytorch.org/t/feedback-on-pytorch-for-kaggle-competitions/2252/4

    def train_test_split(self, **options):
        """ Return a list of splitted indices from a DataSet.
        Indices can be used with DataLoader to build a train and validation set.

        Arguments:
            A Dataset
            A test_size, as a float between 0 and 1 (percentage split) or as an int (fixed number split)
            Shuffling True or False
            Random seed
        """
    
        df_train, df_test = sklearn.model_selection.train_test_split(self.data_df, **options)
        dset_train = NucleusDataset(transform=self.transform)
        dset_train.data_df = df_train
    
        dset_test = NucleusDataset(transform=self.transform)
        dset_test.data_df = df_test
    
        return dset_train, dset_test
    
    #length = len(dataset)
    #indices = list(range(1, length))

    #if shuffle:
    #    random.seed(random_seed)
    #    random.shuffle(indices)

    #if isinstance(test_size, float):
    #    split = int(math.floor(test_size * length + 0.5))
    #elif isinstance(test_size, int):
    #    split = test_size
    #else:
    #    raise ValueError('%s should be an int or a float' % str)
    #return indices[split:], indices[:split]



Overwriting dataset.py


In [326]:
import transform
from transform import random_rotate90_transform2
import torchvision
from torchvision.transforms import ToTensor, ToPILImage

#Note: for image and mask, there is no compatible solution that can use transforms.Compse(), see https://github.com/pytorch/vision/issues/9
#transformations = transforms.Compose([random_rotate90_transform2(),transforms.ToTensor(),])

def train_transform(img, mask):
    #print img.shape
    #print mask.shape
    #print type(img)
    img, mask = random_rotate90_transform2(img, mask, 0.5)
    #print 'after'
    #print img.shape
    #print mask.shape
    #print type(mask)
    # HACK
    if len(mask.shape) == 2:
      mask = np.expand_dims(mask, 2)
    #img = torch.from_numpy(np.transpose(img,(2,0,1))).float() 
    #mask = torch.from_numpy(np.transpose(mask,(2,0,1))).float()
    img = TT.ToTensor()(img)
    mask = TT.ToTensor()(mask)
    return img, mask

dsb_data_dir = os.path.join('..', '..', 'input')
stage_name = 'stage1'
#dset = NucleusDataset(dsb_data_dir, stage_name,transform=train_transform)

#dset_save = dset
#train_idx, valid_idx = train_valid_split(dset,test_size=0.05,random_seed=1,shuffle=True)
#train_sampler = SubsetRandomSampler(train_idx)
#valid_sampler = SubsetRandomSampler(valid_idx)
#train_loader = DataLoader(dset,batch_size=1,sampler=train_sampler,num_workers=4)
#valid_loader = DataLoader(dset,batch_size=1,sampler=valid_sampler,num_workers=4)

# hack: this image format (1388, 1040) occurs only ones, stratify complains .. 
#dset.data_df.drop(dset.data_df[dset.data_df['size'] == (1388, 1040)].index[0])
dset.data_df = dset.data_df[dset.data_df['size'] != (1388, 1040)]
#idx = np.where(dset.data_df['size'] == (1388, 1040))

stratify = dset.data_df['images'].map(lambda x: '{}'.format(x.size))
train_dset, valid_dset = dset.train_test_split(test_size=0.05, random_state=1, shuffle=True, stratify=stratify)
train_loader = DataLoader(train_dset, batch_size=1,shuffle=True)
valid_loader = DataLoader(valid_dset, batch_size=1,shuffle=True)



In [87]:
# summary of image formats
dset.data_df.groupby(['format','mode','size']).size()

format  mode  size        
PNG     RGBA  (256, 256)      334
              (320, 256)      112
              (347, 260)        5
              (360, 360)       91
              (640, 512)       13
              (696, 520)       92
              (1024, 1024)     16
              (1272, 603)       6
              (1388, 1040)      1
dtype: int64

In [40]:
modes = dset.data_df['images'].map(lambda x: x.mode)
sizes = dset.data_df['images'].map(lambda x: x.size)
print np.unique(sizes)

[(256, 256) (320, 256) (347, 260) (360, 360) (640, 512) (696, 520)
 (1024, 1024) (1272, 603) (1388, 1040)]


In [309]:
#%%writefile architectures.py
#%pycat architectures.py

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

# class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)[source]


class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, stride=1, kernel_size=3, padding=1),
            #nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=1, padding=1))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 16, stride=1, kernel_size=3, padding=1),
            #nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=1, padding=1))
        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 16, stride=1, kernel_size=3, padding=1),
            #nn.BatchNorm2d(16),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(16, 1, stride=1, kernel_size=1, padding=0),
            nn.Sigmoid())

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        return out



In [283]:
#%pycat main.py
#%%writefile main.py


import os
import shutil
import torch
from torch.autograd import Variable


def save_model(
        model,
        optimizer,
        stats,
        it,
        is_best=False,
        fname='model_save.pth.tar'):
    m_state_dict = model.state_dict()
    o_state_dict = optimizer.state_dict()

    torch.save({
        'it': it,
        'model_state_dict': m_state_dict,
        'optimizer_state_dict': o_state_dict,
        'stats': stats},
        fname)
    if is_best:
        shutil.copyfile(fname, 'model_best.pth.tar')


def load_model(model, optimizer, fname='model_best.pth.tar'):

    if os.path.isfile(fname):
        checkpoint = torch.load(fname)
        it = checkpoint['it']
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        stats = checkpoint['stats']
        print(
            "=> loaded checkpoint '{}' (epoch {})".format(
                fname, checkpoint['epoch']))
        return it, stats
    else:
        print("=> no checkpoint found at '{}'".format(fname))
        return None, None


def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    cnt = 0
    for i, (img, labels) in enumerate(loader):
        img, labels = Variable(img), Variable(labels)
        outputs = model(img)
        loss = criterion(outputs, labels)
        running_loss += loss.data[0]
        cnt = cnt + 1
    l = running_loss / cnt
    return l


def train(
        train_loader,
        valid_loader,
        model,
        criterion,
        optimizer,
        stats,
        best_loss,
        best_it,
        epoch,
        it,
        eval_every,
        print_every,
        save_every):
    running_loss = 0.0
    cnt = 0
    for i, (img, labels) in enumerate(train_loader, it + 1):
        img, labels = Variable(img), Variable(labels)
        optimizer.zero_grad()
        model.train(True)

        outputs = model(img)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        cnt = cnt + 1

        running_loss += loss.data[0]
        if cnt > 0 and i % eval_every == 0:

            l = validate(model, valid_loader, criterion)
            stats.append((i, running_loss / cnt, l))
            running_loss = 0.0

            if cnt > 0 and i % print_every == 0:
                print('[%d, %d]\ttrain loss: %.3f\tvalid loss: %.3f' %
                      (epoch, i, stats[-1][1], stats[-1][2]))
            if i % save_every == 0:
                is_best = False
                if best_loss > l:
                    best_loss = l
                    best_it = i
                    is_best = True
                    save_model(
                        model,
                        optimizer,
                        stats,
                        i,
                        is_best,
                        'model_save.pth.tar')
    return i, best_loss, best_it


def adjust_learning_rate(optimizer, epoch, lr0):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = lr0 * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr



In [323]:
from torch import optim
model = CNN()



criterion = nn.MSELoss()
#criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001,momentum=0.9, weight_decay=1e-4)
#optimizer = optim.Adam(model.parameters(),lr=0.0001,weight_decay=1e-4)


print_every = 10
save_every = 10
eval_every = 10
    
epochs = 10

it = 0
best_loss = 1e20
best_it = 0
stats = []
for epoch in range(epochs):
    # adjust_learning_rate(optimizer, epoch)
    it, best_loss, best_it = train(train_loader, valid_loader, model, criterion, optimizer, stats, best_loss, best_it, epoch, it, eval_every, print_every, save_every)



AttributeError: 'module' object has no attribute 'to_tensor'

In [328]:
import torchvision.transforms as TT
#monkeypatch(dset, 'transform', train_transform)
train_dset.transform=train_transform
valid_dset.transform=train_transform
#for i,(img,mask) in enumerate(train_loader):
#    break

#img = F.to_pil_image(img)
#mask = F.to_pil_image(mask)
print img.shape, mask.shape, img.min(), img.max(), mask.min(), mask.max()
#model.eval()
#print type(img)
#pred = model(Variable(img,requires_grad=False))
#show_img([img,mask,pred.data])
#plot_img_and_hist(mask)
mask.mean()
mask.shape
#show_img([img,mask])
#train_dset.transform
#train_dset.transform=transforms.ToTensor()
#train_dset.transform
TT.ToPILImage()(img)

torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256]) 2.0 123.0 0.0 255.0


ValueError: axes don't match array

In [143]:
epoch,stats=load_model(model, optimizer)
print epoch, stats

KeyError: 'it'

In [314]:
dir(F)

['__all__',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '_range',
 'btriunpack',
 'chunk',
 'matmul',
 'mul',
 'reduce',
 'split',
 'stack',
 'torch',
 'unbind']